In [4]:
!pip install pyodbc

In [13]:
import pandas as pd

### Creat a connection to the Database

In [1]:
import pyodbc

# Define the connection string with encryption disabled
connection_string = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=SQL\\SQLEXPRESS;"
    "DATABASE=HDB;"
    "Trusted_Connection=yes;"
    "Encrypt=no;"
)

# Establish a connection to the database
connection = pyodbc.connect(connection_string)

# Create a cursor object using the connection
cursor = connection.cursor()

# Test the connection by fetching some data
cursor.execute("SELECT @@version;")
row = cursor.fetchone()
print(row)

('Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) \n\tOct  8 2022 05:58:25 \n\tCopyright (C) 2022 Microsoft Corporation\n\tExpress Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 22631: ) (Hypervisor)\n',)


In [20]:
# Set Commit to be true by default

connection.autocommit=True

## Create Table Student

In [22]:
try:
    cursor.execute("CREATE TABLE STUDENTS( student_id int PRIMARY KEY, name varchar(100), age int, gender varchar(10), subject varchar(50), marks int);")

except pyodbc.Error as e:
    print("Error creating table")
    print(e)

In [18]:
!pip install faker

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.8 MB 1.7 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/1.8 MB 1.7 MB/s eta 0:00:01
   ----- ---------------------------------- 0.3/1.8 MB 2.0 MB/s eta 0:00:01
   ------ --------------------------------- 0.3/1.8 MB 1.7 MB/s eta 0:00:01
   ------ --------------------------------- 0.3/1.8 MB 1.7 MB/s eta 0:00:01
   ------ --------------------------------- 0.3/1.8 MB 1.7 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.8 MB 967.8 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.8 MB 873.8 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.8 MB 873.8 kB/s eta 0:00:02
   ------- -------------------------------- 0.4/1.8 MB 768.6 kB/s eta 0:00:02
   -------- ------------------------------- 0.4/1.8 MB 740.5 kB/s eta 0:00:02
   -------- ----

In [23]:
import random
from faker import Faker

# Create a Faker instance to generate random data
fake = Faker()

# Insert 10 random records into the STUDENTS table
for _ in range(10):
    student_id = random.randint(1, 1000)  # Random student_id between 1 and 1000
    name = fake.name()                   # Random name
    age = random.randint(18, 25)         # Random age between 18 and 25
    gender = random.choice(['Male', 'Female'])  # Random gender

    query = f"""
    INSERT INTO STUDENTS (student_id, name, age, gender, subject, marks) 
    VALUES ({student_id}, '{name}', {age}, '{gender}');
    """

    try:
        cursor.execute(query)
    except pyodbc.Error as e:
        print(f"Error inserting data: {e}")

print("Data inserted successfully.")

Data inserted successfully.


In [12]:
## Check Values are inserted in STUDENTS table

try:
    cursor.execute("SELECT * FROM STUDENTS")
except pyodbc.Error as e:
        print(f"Error inserting data: {e}")     

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(65, 'Bailey Johnson', 20, 'Male')
(143, 'Nicholas Flores', 18, 'Female')
(146, 'Charles West', 22, 'Male')
(406, 'Antonio Perkins', 18, 'Male')
(500, 'Paul Carlson', 24, 'Female')
(554, 'Dr. Andrea Vasquez DDS', 18, 'Female')
(588, 'Richard Smith', 18, 'Female')
(657, 'Eric Johnston', 25, 'Male')
(958, 'Catherine Mccarthy', 23, 'Male')
(987, 'Mrs. Andrea Martin PhD', 19, 'Male')


## Read CSV files

In [14]:
Courses = pd.read_csv(r"C:\Users\omara\OneDrive\Desktop\Omar\Portfolio\Data Engineer\Project 1\Courses.csv")

In [16]:
Courses.head()

,course_id,course_name
0,1,Science
1,2,English
2,3,History
3,4,Math


In [17]:
Marks = pd.read_csv(r"C:\Users\omara\OneDrive\Desktop\Omar\Portfolio\Data Engineer\Project 1\Marks.csv")

In [18]:
Marks.head()

,student_id,course_id,time_study,marks
0,65,3,4.508,19.202
1,143,4,0.096,7.734
2,146,4,3.133,13.811
3,406,3,7.909,53.018
4,500,3,7.811,55.299


## Create Courses and Marks tables

In [24]:
try:
   # Create the COURSES table
    cursor.execute("""
        CREATE TABLE COURSES (
            course_id int PRIMARY KEY,
            course_name varchar(100)
        );
    """)

    # Create the MARKS table with foreign keys referencing the COURSES and STUDENTS tables
    cursor.execute("""
        CREATE TABLE MARKS (
            student_id int,
            course_id int,
            time_study float,
            marks float,
            FOREIGN KEY (course_id) REFERENCES COURSES(course_id),
            FOREIGN KEY (student_id) REFERENCES STUDENTS(student_id)
        );
    """)

except pyodbc.Error as e:
    print("Error creating table")
    print(e)

## Insert Courses and Marks csv files into COURSES & MARKS tables

In [ ]:
## Insert data into the COURSES table

try:
    for index, row in Courses.iterrows():
        cursor.execute("""
            INSERT INTO COURSES (course_id, course_name)
            VALUES (?, ?)
        """, row['course_id'], row['course_name'])

except pyodbc.Error as e:
    print("Error inserting data")
    print(e)

In [31]:
## Insert data into the MARKS table

try:
    for index, row in Marks.iterrows():
        cursor.execute("""
            INSERT INTO MARKS (student_id, course_id, time_study, marks)
            VALUES (?, ?, ?, ?)
        """, row['student_id'], row['course_id'], row['time_study'], row['marks'])

except pyodbc.Error as e:
    print("Error inserting data")
    print(e)

In [35]:
## Check Values are inserted in MARKS table

try:
    cursor.execute("SELECT * FROM MARKS")
except pyodbc.Error as e:
        print(f"Error inserting data: {e}")     

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(65, 3, 4.508, 19.202)
(143, 4, 0.096, 7.734)
(146, 4, 3.133, 13.811)
(406, 3, 7.909, 53.018)
(500, 3, 7.811, 55.299)
(554, 2, 3.211, 17.822)
(588, 4, 6.063, 29.889)
(657, 1, 3.413, 17.264)
(958, 1, 4.41, 20.348)
(987, 2, 6.173, 30.862)
(65, 4, 6.508, 50.202)
(500, 1, 2.811, 45.789)


In [36]:
## Check Values are inserted in COURSES table

try:
    cursor.execute("SELECT * FROM COURSES")
except pyodbc.Error as e:
        print(f"Error inserting data: {e}")     

row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

(1, 'Science')
(2, 'English')
(3, 'History')
(4, 'Math')


## Create Indexes And Logging To Capture Connections, Table Creations, Index Creations, And Errors

In [6]:
import logging

# Configure logging
logging.basicConfig(filename='database_operations.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# SQL commands to create indexes
create_index_queries = {
    'MARKS': "CREATE INDEX idx_student_course ON MARKS(student_id, course_id);",
    'COURSES': "CREATE INDEX idx_course_id ON COURSES(course_id);",
    'STUDENTS': "CREATE INDEX idx_student_id ON STUDENTS(student_id);"}

try:
    logging.info('Starting index creation process.')

    # Create indexes for each table as specified
    for table, query in create_index_queries.items():
        try:
            logging.info(f'Creating index for {table}...')
            cursor.execute(query)
            connection.commit()
            logging.info(f'Index created successfully for {table}.')
        except pyodbc.Error as e:
            logging.error(f'Error creating index for {table}: {e}')

except pyodbc.Error as e:
    logging.Error('Database error occurred during index creation: %s', e)
except Exception as e:
    logging.Error('Unexpected error occurred during index creation: %s', e)
finally:
    logging.info('Index creation process completed.')